<a href="https://colab.research.google.com/github/uddhavp22/Mamba-BCI-Journal-of-Neural-Engineering/blob/main/Benchmakring%20Braindecode_MOABB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install lightning
!pip install braindecode
!pip install moabb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 808.5/808.5 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.3/812.3 kB 22.9 MB/s eta 0:00:00


# Loading Data structure

In [30]:
import braindecode
from braindecode.datasets import MOABBDataset

subject_id = 3
dataset = MOABBDataset(dataset_name="Stieger2021", subject_ids=[subject_id])


  0%|                                               | 0.00/650M [00:00<?, ?B/s]
  0%|                                      | 1.17M/650M [00:00<00:58, 11.0MB/s]
  1%|▎                                     | 5.76M/650M [00:00<00:20, 31.1MB/s]
  2%|▋                                     | 11.3M/650M [00:00<00:15, 42.2MB/s]
  3%|█                                     | 17.2M/650M [00:00<00:13, 48.6MB/s]
  3%|█▎                                    | 22.3M/650M [00:00<00:12, 49.5MB/s]
  4%|█▌                                    | 27.6M/650M [00:00<00:12, 50.5MB/s]
  5%|█▉                                    | 32.7M/650M [00:00<00:12, 50.7MB/s]
  6%|██▏                                   | 37.8M/650M [00:00<00:12, 47.6MB/s]
  7%|██▍                                   | 42.6M/650M [00:00<00:14, 41.9MB/s]
  7%|██▊                                   | 47.7M/650M [00:01<00:13, 44.4MB/s]
  8%|███                                   | 52.7M/650M [00:01<00:12, 46.0MB/s]
  9%|███▎                              

ValueError: This should be the cue time-point,

# Preprocessing

In [ ]:
import numpy as np

from braindecode.preprocessing import (
    exponential_moving_standardize,
    preprocess,
    Preprocessor,
)

low_cut_hz = 4.0  # low cut frequency for filtering
high_cut_hz = 38.0  # high cut frequency for filtering
# Parameters for exponential moving standardization
factor_new = 1e-3
init_block_size = 1000

transforms = [
    Preprocessor("pick_types", eeg=True, meg=False, stim=False),  # Keep EEG sensors
    Preprocessor(
        lambda data, factor: np.multiply(data, factor),  # Convert from V to uV
        factor=1e6,
    ),
    Preprocessor("filter", l_freq=low_cut_hz, h_freq=high_cut_hz),  # Bandpass filter
    Preprocessor(
        exponential_moving_standardize,  # Exponential moving standardization
        factor_new=factor_new,
        init_block_size=init_block_size,
    ),
]

# Transform the data
preprocess(dataset, transforms, n_jobs=-1)

/usr/local/lib/python3.10/dist-packages/braindecode/preprocessing/preprocess.py:55: UserWarning: Preprocessing choices with lambda functions cannot be saved.
  warn('Preprocessing choices with lambda functions cannot be saved.')


# Cut Compute Windows

In [ ]:
from braindecode.preprocessing import create_windows_from_events

trial_start_offset_seconds = -0.5
# Extract sampling frequency, check that they are same in all datasets
sfreq = dataset.datasets[0].raw.info["sfreq"]
assert all([ds.raw.info["sfreq"] == sfreq for ds in dataset.datasets])
# Calculate the trial start offset in samples.
trial_start_offset_samples = int(trial_start_offset_seconds * sfreq)

# Create windows using braindecode function for this. It needs parameters to define how
# trials should be used.
windows_dataset = create_windows_from_events(
    dataset,
    trial_start_offset_samples=trial_start_offset_samples,
    trial_stop_offset_samples=0,
    preload=True,
)

Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']
Used Annotations descriptions: ['feet', 'left_hand', 'right_hand', 'tongue']


# Create Pytorch model

In [ ]:
import torch
from braindecode.models import ShallowFBCSPNet
from braindecode.util import set_random_seeds

cuda = torch.cuda.is_available()  # check if GPU is available, if True chooses to use it
device = "cuda" if cuda else "cpu"
if cuda:
    torch.backends.cudnn.benchmark = True
seed = 20200220
set_random_seeds(seed=seed, cuda=cuda)

n_classes = 4
classes = list(range(n_classes))
# Extract number of chans and time steps from dataset
n_channels = windows_dataset[0][0].shape[0]
input_window_samples = windows_dataset[0][0].shape[1]

# The ShallowFBCSPNet is a `nn.Sequential` model

model = ShallowFBCSPNet(
    n_channels,
    n_classes,
    input_window_samples=input_window_samples,
    final_conv_length="auto",
)

# Display torchinfo table describing the model
print(model)

# Send model to GPU
if cuda:
    model.cuda()

/usr/local/lib/python3.10/dist-packages/braindecode/util.py:51: UserWarning: torch.backends.cudnn.benchmark was set to True which may results in lack of reproducibility. In some cases to ensure reproducibility you may need to set torch.backends.cudnn.benchmark to False.
  warn(
/usr/local/lib/python3.10/dist-packages/braindecode/models/base.py:23: UserWarning: ShallowFBCSPNet: 'input_window_samples' is depreciated. Use 'n_times' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/braindecode/models/base.py:180: UserWarning: LogSoftmax final layer will be removed! Please adjust your loss function accordingly (e.g. CrossEntropyLoss)!
  warnings.warn("LogSoftmax final layer will be removed! " +


Layer (type (var_name):depth-idx)        Input Shape               Output Shape              Param #                   Kernel Shape
ShallowFBCSPNet (ShallowFBCSPNet)        [1, 22, 1125]             [1, 4]                    --                        --
├─Ensure4d (ensuredims): 1-1             [1, 22, 1125]             [1, 22, 1125, 1]          --                        --
├─Rearrange (dimshuffle): 1-2            [1, 22, 1125, 1]          [1, 1, 1125, 22]          --                        --
├─CombinedConv (conv_time_spat): 1-3     [1, 1, 1125, 22]          [1, 40, 1101, 1]          36,240                    --
├─BatchNorm2d (bnorm): 1-4               [1, 40, 1101, 1]          [1, 40, 1101, 1]          80                        --
├─Expression (conv_nonlin_exp): 1-5      [1, 40, 1101, 1]          [1, 40, 1101, 1]          --                        --
├─AvgPool2d (pool): 1-6                  [1, 40, 1101, 1]          [1, 40, 69, 1]            --                        [75, 1]
├─Express

# EEGNet Run

In [ ]:

from braindecode.models import EEGNetv4

model=EEGNetv4(n_chans=22, n_outputs=4, n_times=1125,)
model.cuda()
input=torch.randn(1,22,1125).cuda()
print(model(input).shape)

torch.Size([1, 4])


# Mamba Run

In [ ]:
import numpy as np

import torch
import torch.nn as nn

from torch import tensor


import scipy as sp
!pip install torch_geometric
import torch_geometric
from torch_geometric import utils
from torch_geometric.utils import dense_to_sparse
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"
import torchvision
import torchvision.transforms as transforms

!pip install mamba-ssm
import torch
from mamba_ssm import Mamba
from torch.utils.data import DataLoader, TensorDataset
from mamba_ssm.modules.block import Block
from functools import partial

In [ ]:
class Mambablock(nn.Module):
    def __init__(self, d_model, n_blocks, d_state=16, expand=2):
        super(Mambablock, self).__init__()
        self.n_blocks = n_blocks
        self.d_model = d_model
        config = {'d_state': d_state, 'expand': expand}

        # Create Mamba instance with d_model as a keyword argument
        self.mamba_blocks = nn.ModuleList([Block(d_model, partial(Mamba,**config), nn.Identity) for _ in range(self.n_blocks)])

    def forward(self, x):
        for block in self.mamba_blocks:
            x, _ = block(x)
        return x

class EEGMamba(nn.Module):
    def __init__(self, num_channels, num_blocks, dim, d_state=256, d_conv=4, expand=2,d_model=1):
        super(EEGMamba, self).__init__()
        # Pass dim as d_model to Mambablock
        self.channel_models = nn.ModuleList([Mambablock(dim, num_blocks, d_state, expand) for _ in range(num_channels)])
        #self.linear = nn.Linear(1001, d_state) if u want a linear transform after data

    def forward(self, x):
        batch_size = x.size(0)
        channel_outputs = []

        # Iterate over each channel model and corresponding channel data
        for i in range(x.size(1)):



            channel_data = x[:, i, :,:]  # Extract data for the i-th channel


            model = self.channel_models[i]
            output = model(channel_data)
            averaged_output = torch.mean(output, dim=1, keepdim=True)
            channel_outputs.append(averaged_output)

        # Stack the channel outputs along the correct dimension
        combined_output = torch.stack(channel_outputs, dim=1)
        return combined_output

class MambaJambaNet(nn.Module):
    def __init__(self, num_channels, num_layers, dim, d_state=16, d_conv=4, expand=2, kernel_size=2, num_classes=4):
        super(MambaJambaNet, self).__init__()
        self.mamba_model = EEGMamba(num_channels, num_layers, dim, d_state, d_conv, expand)
        self.classifier = nn.Sequential(
            nn.Linear(num_channels, 46),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(46, num_classes)
        )

    def forward(self, x):
        #print(x.shape)
        x=torch.unsqueeze(x,-1)
        x = self.mamba_model(x)

        x = x.squeeze(2)  # making it (batch_size, num_channels, dim)
        x = x.squeeze(2)
        x = self.classifier(x)
        return x

input_tensor = torch.randn(1, 22, 1001).to("cuda")
num_channels = 22  # Define num_channels
num_layers = 4  # Define num_layers, choose as needed
dim = 1 # Define dim, choose as needed
model = MambaJambaNet(num_channels=22, num_layers=4, dim=1).to("cuda")
print(model(input_tensor).shape)

#rn total parameters - 11,888

torch.Size([1, 4])


# Split dataset into train and test

In [ ]:
splitted = windows_dataset.split("session")
train_set = splitted['0train']  # Session train
test_set = splitted['1test']  # Session evaluation

# Train with lightning

In [ ]:
from torch.nn import Module
from torch.optim.lr_scheduler import LRScheduler
from torch.utils.data import DataLoader

lr = 0.0625 * 0.01
weight_decay = 0
batch_size = 1
n_epochs = 20
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False)

In [ ]:
import lightning as L
from torchmetrics.functional import accuracy
from lightning.pytorch import Trainer
from lightning.pytorch.loggers import TensorBoardLogger

logger = TensorBoardLogger("tb_logs", name="my_model")
trainer = Trainer(logger=logger)

class LitModule(L.LightningModule):
    def __init__(self, module):
        super().__init__()
        self.module = module
        self.loss = nn.CrossEntropyLoss()

    def training_step(self, batch, batch_idx):
        x, y, _ = batch
        y_hat = self.module(x)
        loss = self.loss(y_hat, y)
        self.log("train_loss", loss)

        return loss

    def test_step(self, batch, batch_idx):
        x, y, _ = batch
        y_hat = self.module(x)
        loss = self.loss(y_hat, y)
        acc = accuracy(y_hat, y, "multiclass", num_classes=4)
        metrics = {"test_acc": acc, "test_loss": loss}
        self.log_dict(metrics)
        return metrics

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(model.parameters(), lr=lr,
                                      weight_decay=weight_decay)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer,
                                                               T_max=n_epochs - 1)
        return [optimizer], [scheduler]


# Creating the trainer with max_epochs=2 for demonstration purposes
trainer = L.Trainer(max_epochs=n_epochs)
# Create and train the LightningModule
lit_model = LitModule(model)
trainer.fit(lit_model, train_loader)

# After training, you can test the model using the test DataLoader
trainer.test(dataloaders=test_loader)

INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: GPU available: True (cuda), used: True
INFO:lightning.pytorch.utilities.rank_zero:GPU available: True (cuda), used: True
INFO: TPU available: False, using: 0 TPU cores
INFO:lightning.pytorch.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO: HPU available: False, using: 0 HPUs
INFO:lightning.pytorch.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: 
  | Name   | Type             | Params | Mode 
----------------------------------------------------
0 | 

Training: |          | 0/? [00:00<?, ?it/s]

INFO: `Trainer.fit` stopped: `max_epochs=100` reached.
INFO:lightning.pytorch.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=100` reached.
INFO: Restoring states from the checkpoint path at /content/lightning_logs/version_6/checkpoints/epoch=99-step=4100.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Restoring states from the checkpoint path at /content/lightning_logs/version_6/checkpoints/epoch=99-step=4100.ckpt
INFO: LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:lightning.pytorch.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO: Loaded model weights from the checkpoint at /content/lightning_logs/version_6/checkpoints/epoch=99-step=4100.ckpt
INFO:lightning.pytorch.utilities.rank_zero:Loaded model weights from the checkpoint at /content/lightning_logs/version_6/checkpoints/epoch=99-step=4100.ckpt


Testing: |          | 0/? [00:00<?, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│         test_acc          │           0.25            │
│         test_loss         │    1.3862948417663574     │
└───────────────────────────┴───────────────────────────┘

[{'test_acc': 0.25, 'test_loss': 1.3862948417663574}]

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir=lightning_logs/ --port=6007


In [ ]:
%tensorboard --logdir=tb_logs/ download


ERROR: Failed to launch TensorBoard (exited with 2).
Contents of stderr:
2024-07-16 01:05:40.772630: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-16 01:05:40.772703: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-16 01:05:40.774171: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-16 01:05:41.786901: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
usage: tensorboard [-h] [--helpfull] [--logdir PATH] [--logdir_spec PATH_SPEC] [--host ADDR]
                   [--bind_all] [--port PORT] [--reuse_port BOOL] [--load_f

In [ ]:
t%tensorboard --logdir=tb_logs download --tag=version_1 --output_dir=downloaded_models

{'test_acc': tensor(0.5868), 'test_loss': tensor(0.9856)}

EEGNET BEST RUN VV